In [8]:
import numpy as np
import scipy as scp
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#Keras Imports
from tensorflow import keras
from tensorflow.keras import backend as K

from tensorflow.keras.datasets import cifar10, mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Layer, Activation, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras import activations

import os
import sys


#Utility
# from utils import binarize
sys.path.insert(0, '..')
from binarization_utils import *
# from model_architectures import get_model

#Probability
import tensorflow_probability as tfp


In [9]:
print(tf.__version__)
print(keras.__version__)

2.6.0
2.6.0


In [10]:
dataset='MNIST'
# Train=True
# Evaluate=False

In [11]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# convert class vectors to binary class matrices
X_train = X_train.reshape(-1,784)
X_test = X_test.reshape(-1,784)

#Restructure Datea
X_train=X_train.astype(np.float32)
X_test=X_test.astype(np.float32)
# Y_train = to_categorical(y_train, 10)
# Y_test = to_categorical(y_test, 10)
X_train /= 255
X_test /= 255
X_train=2*X_train-1
X_test=2*X_test-1


#Train Val Split
X_train,X_val,y_train, y_val = train_test_split(X_train,y_train,test_size = 0.10)
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(y_train.shape[0], 'train samples')

print(X_val.shape[0], 'val samples')
print(y_val.shape[0], 'val samples')

print(X_test.shape[0], 'test samples')
print(y_test.shape[0], 'test samples')


NameError: name 'train_test_split' is not defined

In [12]:

# def computeP(n,k, mu_L, mu_H, var_L, var_H, threshold):
#     """
#     Computes probability of error for a specific input with final inner product k
    
#     n : Length of vector
#     k : input
#     mu : Mean
#     var : Variance
    
#     _h: High state
#     _l: low state
#     """

#     p01 = []
#     p10 = []
#     means = [m * mu_L + (n_dists - 1 - m) * mu_H for m in range(0, n_dists)]
#     variances = [m**2 * var_L + (n_dists - 1 - m)**2 * var_H for m in range(0, n_dists)]
#     for t in range(0,n_dists):
#         if t < n_dists - 1 - t:
#             p01 = p01 + [1 - norm.cdf(threshold, means[t], np.sqrt(variances[t]))]
#         if t > n_dists - 1 - t:
#             p10 = p10 + [norm.cdf(threshold, means[t], np.sqrt(variances[t]))]
#     return p01, p10


class Sign_layer(Layer):
    def __init__(self, levels=1,**kwargs):
        self.levels=levels
        super(Sign_layer, self).__init__(**kwargs)
    def build(self, input_shape):
        ars=np.arange(self.levels)+1.0
        ars=ars[::-1]
        means=ars/np.sum(ars)
        self.means=[K.variable(m) for m in means]
        self._trainable_weights = self.means
        
    def call(self, x, mask=None):
        resid = x
        out_bin=0
        for l in range(self.levels):
            out=binarize(resid)*(K.abs(self.means[l]))
#             print(out)
            out_bin=out_bin+out
            resid=resid-out
        return out_bin
    
        # the following lines were an idea to implement flips using tensor operations
        '''positive_mask = tf.cast(out_bin > 0, tf.float32)
        negative_mask = tf.cast(out_bin < 0, tf.float32)
        
        positive_flips = tf.random.uniform(out_bin.shape) < p[1]
        positives = tf.math.multiply(positive_mask, (tf.cast(tf.random.uniform(out_bin.shape) < p[1], tf.float32) - 1))
        negatives = tf.math.multiply(negative_mask, (tf.cast(tf.random.uniform(out_bin.shape) < p[0], tf.float32) - 1))
        return'''

    def get_output_shape_for(self,input_shape):
        return input_shape
    def compute_output_shape(self,input_shape):
        return input_shape
    def set_means(self,X):
        means=np.zeros((self.levels))
        means[0]=1
        resid=np.clip(X,-1,1)
        approx=0
        for l in range(self.levels):
            m=np.mean(np.absolute(resid))
            out=np.sign(resid)*m
            approx=approx+out
            resid=resid-out
            means[l]=m
            err=np.mean((approx-np.clip(X,-1,1))**2)

        means=means/np.sum(means)
        sess=K.get_session()
        sess.run(self.means.assign(means))
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'levels': self.levels
        })
        return config

class binary_dense_error_var(Layer):
    def __init__(self,n_in,n_out,error_type,error_dict,**kwargs):
        self.n_in=n_in
        self.n_out=n_out
        self.error_type=error_type
        self.error_dict= error_dict
        assert self.error_type in ["NoError", "Flip","InnerDot"], "error_type of {0} is invalid".format(error_type)
        super(binary_dense_error_var,self).__init__(**kwargs)
        
        if(error_type == "InnerDot"):
            max_pop = self.n_in
            self.uh = self.error_dict["uh"]
            self.ul = self.error_dict["ul"]
            self.sigl = self.error_dict["sigl"]
            self.sigh = self.error_dict["sigh"]

            #Not Used, Mainly used for information
            means = [m * self.ul + (max_pop - m) * self.uh for m in range(0, max_pop+1)]
            stdvs = [np.sqrt(m * (self.sigl**2) + (max_pop - m)*(self.sigh**2)) for m in range(0, max_pop+1)]
            self.distributions = [tfp.distributions.Normal(loc= m, scale = sig) for m,sig in zip(means,stdvs)]
    
        elif(error_type == "Flip"):
            self.p = self.error_dict["p"]
            
    def build(self, input_shape):
        stdv=1/np.sqrt(self.n_in)
        w = np.random.normal(loc=0.0, scale=stdv,size=[self.n_in,self.n_out]).astype(np.float32)
        self.w=K.variable(w)
        self.gamma_w=K.variable(1.0)
        self._trainable_weights=[self.w,self.gamma_w]
        
        
#         if(error_type == "InnerDot"):
#             max_pop = self.n_in+1
#             self.uh = self.error_dict["uh"]
#             self.ul = self.error_dict["ul"]
#             self.sigl = self.error_dict["sigl"]
#             self.sigh = self.error_dict["sigh"]

#             means = [m * ul + (max_pop - 1 - m) * uh for m in range(0, max_pop+1)]
#             stdvs = [np.sqrt(m * (sigl**2) + (max_pop - 1 - m)*(sigh**2)) for m in range(0, max_pop+1)]
#             self.distributions = [tfp.distributions.Normal(loc= m, scale = sig) for m,sig in zip(means,stdvs)]
            
    def call(self, x,training=None):
        
        #Designed with Batch norm in mind, Binarization done after. 
        if(self.error_type == "NoError"):
            self.clamped_w=binarize(self.w)
            self.prod=K.abs(self.gamma_w)*K.dot(x,self.clamped_w)
            self.out = self.prod
            return self.out
        elif(self.error_type == "Flip"):
            self.clamped_w=binarize(self.w)*((2*tf.cast(tf.random.uniform(self.w.shape) > self.p, tf.float32)) - 1)
            self.prod=K.abs(self.gamma_w)*K.dot(x,self.clamped_w)
            self.out = self.prod
            return self.out
        elif(self.error_type == "InnerDot"):
            
            if training:
                
                #Perfrom error model on inference
                #No Error Model first
                self.clamped_w=binarize(self.w)
                
                self.prod=K.abs(self.gamma_w)*K.dot(x,self.clamped_w)
                
                # Make Model
                self.clamped_x=binarize(x)
                gamma_x = tf.math.maximum(K.abs(tf.math.reduce_min(x)),K.abs(tf.math.reduce_max(x)))
                #self.prod = K.dot(x,self.clamped_w)
                self.pop = (K.dot(self.clamped_x,self.clamped_w)+self.n_in)/2

                #Add Error 
                self.dist = tfp.distributions.Normal(loc = self.pop * self.ul + (self.n_in - self.pop) * self.uh,
                                                     scale= tf.math.sqrt(self.pop * (self.sigl**2) + (self.n_in - self.pop)*(self.sigh**2)))
                
                self.samps = self.dist.sample(1)
                self.samps = tf.squeeze(self.samps, axis = 0)
        
                #Use Histogram for binning
                self.step = self.ul - self.uh
                self.edges = np.arange(self.n_in*self.uh-self.step/2, self.n_in*self.ul+self.step, self.step,dtype=np.float32)
                self.bins= tfp.stats.find_bins(self.samps, self.edges, extend_lower_interval = True, extend_upper_interval=True)
                self.out = K.abs(self.gamma_w)*gamma_x*(2*self.bins-self.n_in)
                
                
                #Gradient not using binning
                return  self.prod + K.stop_gradient(self.out -  self.prod)

            else:
                #Perfrom error model on inference
                #No Error Model first
                self.clamped_w=binarize(self.w)
                self.clamped_x=binarize(x)
                gamma_x = tf.math.maximum(K.abs(tf.math.reduce_min(x)),K.abs(tf.math.reduce_max(x)))
                #self.prod = K.dot(x,self.clamped_w)
                self.pop = (K.dot(self.clamped_x,self.clamped_w)+self.n_in)/2

                #Add Error 
                self.dist = tfp.distributions.Normal(loc = self.pop * self.ul + (self.n_in - self.pop) * self.uh,
                                                     scale= tf.math.sqrt(self.pop * (self.sigl**2) + (self.n_in - self.pop)*(self.sigh**2)))
                

                self.samps = self.dist.sample(1)
                self.samps = tf.squeeze(self.samps, axis = 0)
        
                #Use Histogram for binning
                self.step = self.ul - self.uh
                self.edges = np.arange(self.n_in*self.uh-self.step/2, self.n_in*self.ul+self.step, self.step,dtype=np.float32)
                self.bins= tfp.stats.find_bins(self.samps, self.edges, extend_lower_interval = True, extend_upper_interval=True)
                self.out = K.abs(self.gamma_w)*gamma_x*(2*self.bins-self.n_in)
                
            
                return self.out
        
        
    
    def get_output_shape_for(self,input_shape):
        return (self.n_in, self.n_out)
    
    
    def compute_output_shape(self,input_shape):
        return (self.n_in, self.n_out)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'n_in': self.n_in,
            'n_out': self.n_out,
            'error_type': self.error_type,
            'error_dict': self.error_dict
        })
        return config

In [13]:
# #Testing the Code
# #Model Parameters
# # 0 : No error, params ignored
# # 1 : Flip error, only p 

# # 2 : InnerProd error
# # error_type = "NoError"
# # error_dict = {}

# #Flip Probabiliy
# p=0.1
# # error_type = "Flip"
# # error_dict = {"p": p }

# #InnerProd paramslayer.call([0,0,0])
# ul = 1
# uh = 0.01
# sigl = 0.01
# sigh = 0.01

# error_type = "InnerDot"
# error_dict = {"ul" : ul , "sigl" : sigl, "uh" : uh, "sigh" : sigh}

# layer = binary_dense_error_var(n_in=3,n_out=1,error_type = error_type, error_dict = error_dict)

# out = layer.__call__(tf.constant([[1,1,-1], [1,1,1],[-1,-1,-1]], dtype = tf.float32), training=False)
# print(layer.clamped_w.numpy())
# print(layer.pop.numpy())
# print(layer.samps.numpy())
# print(layer.edges)
# print(layer.bins.numpy())
# print(out.numpy())

In [14]:
#Model Construction

def make_MNIST_model(error_type, p = 0.1, ul = 1, sigl = 0.01, uh=0.01, sigh = 0.01 ):
    if(error_type == 0):
        #No Error
        error_type = "NoError"
        error_dict = {}
    elif(error_type == 1):
        #Flip Error
        error_type = "Flip"
        error_dict = {"p": p }
    elif(error_type == 2):
        #InnerDot error 
        error_type = "InnerDot"
        error_dict = {"ul" : ul , "sigl" : sigl, "uh" : uh, "sigh" : sigh}
        
    #Fixed Parameters 
    batch_norm_eps=1e-4
    batch_norm_alpha=0.1#(this is same as momentum)
    
    #Model Def
    #5 Binary dense layers,final layer is softmax
    model=Sequential()
    #First Layer Group
    model.add(binary_dense_error_var(
        n_in=784,n_out=256,input_shape=[784],error_type = error_type, error_dict = error_dict))
    model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
    model.add(Sign_layer(levels=1))

    #Subsequent Dense Layers
    #Change Layer Sizes
#     layer_sizes = [1024,1024]
    layer_sizes = [256,256,256]
    for i in range(len(layer_sizes)):
        model.add(binary_dense_error_var(
            n_in=int(model.output.get_shape()[1]),n_out=layer_sizes[i],error_type = error_type, error_dict = error_dict))
        model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
        model.add(Sign_layer(levels=1))

    
    #Final Layer
    model.add(binary_dense_error_var(
            n_in=int(model.output.get_shape()[1]),n_out=10,error_type = error_type, error_dict = error_dict))
    model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
    model.add(Activation('softmax'))
    
    # the following is a workaround so that the model weights can be saved
    # https://github.com/tensorflow/tensorflow/issues/46871
    j = 0
    for w in model.weights:
        w._handle_name = 'model_' + str(j) + w.name
        j = j + 1
        
    
    return model

def make_CIFAR10_model(error_type, p = 0.1, ul = 1, sigl = 0.01, uh=0.01, sigh = 0.01 ):
    if(error_type == 0):
        #No Error
        error_type = "NoError"
        error_dict = {}
    elif(error_type == 1):
        #Flip Error
        error_type = "Flip"
        error_dict = {"p": p }
    elif(error_type == 2):
        #InnerDot error 
        error_type = "InnerDot"
        error_dict = {"ul" : ul , "sigl" : sigl, "uh" : uh, "sigh" : sigh}
        
    #Fixed Parameters 
    batch_norm_eps=1e-4
    batch_norm_alpha=0.1#(this is same as momentum)
    
    #Model Def
    model=Sequential()
    model.add(binary_conv(nfilters=64,ch_in=3,k=3,padding='valid',input_shape=[32,32,3]))
    model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
    model.add(Sign_layer(levels=resid_levels))
    model.add(binary_conv(nfilters=64,ch_in=64,k=3,padding='valid'))
    model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
    model.add(Sign_layer(levels=resid_levels))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))

    model.add(binary_conv(nfilters=128,ch_in=64,k=3,padding='valid'))
    model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
    model.add(Sign_layer(levels=resid_levels))
    model.add(binary_conv(nfilters=128,ch_in=128,k=3,padding='valid'))
    model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
    model.add(Sign_layer(levels=resid_levels))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))

    model.add(binary_conv(nfilters=256,ch_in=128,k=3,padding='valid'))
    model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
    model.add(Sign_layer(levels=resid_levels))
    model.add(binary_conv(nfilters=256,ch_in=256,k=3,padding='valid'))
    model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
    model.add(Sign_layer(levels=resid_levels))
    #model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))

    model.add(my_flat())

    model.add(binary_dense_error_var(n_in=int(model.output.get_shape()[1]),n_out=512,error_type = error_type, error_dict = error_dict))
    model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
    model.add(Sign_layer(levels=resid_levels))
    model.add(binary_dense_error_var(n_in=int(model.output.get_shape()[1]),n_out=512,error_type = error_type, error_dict = error_dict))
    model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
    model.add(Sign_layer(levels=resid_levels))
    model.add(binary_dense_error_var(n_in=int(model.output.get_shape()[1]),n_out=10,error_type = error_type, error_dict = error_dict))
    model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
    model.add(Activation('softmax'))
    
    # the following is a workaround so that the model weights can be saved
    # https://github.com/tensorflow/tensorflow/issues/46871
    j = 0
    for w in model.weights:
        w._handle_name = 'model_' + str(j) + w.name
        j = j + 1
        
    
    return model

# Training Script

In [44]:
#Model Parameters
# 0 : No error, params ignored
# 1 : Flip error, only p used
# 2 : InnerProd error
error_type = 2

#Flip Probabiliy
p=0.1

#InnerProd params
ul = 1 #l-low means bits match
uh = 0.1
sigl = 0.01
sigh = 0.2

#Make Model
model = make_MNIST_model(error_type, p = p, ul = ul, sigl = sigl, uh=uh, sigh = sigh )

model.summary()

Model: "sequential_280"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
binary_dense_error_var_1385  (None, 256)               200705    
_________________________________________________________________
batch_normalization_1385 (Ba (None, 256)               1024      
_________________________________________________________________
sign_layer_1108 (Sign_layer) (None, 256)               1         
_________________________________________________________________
binary_dense_error_var_1386  (None, 256)               65537     
_________________________________________________________________
batch_normalization_1386 (Ba (None, 256)               1024      
_________________________________________________________________
sign_layer_1109 (Sign_layer) (None, 256)               1         
_________________________________________________________________
binary_dense_error_var_1387  (None, 256)            

In [45]:
#Training Script
    
#Training parameters
batch_size=50
epochs=100

#Makes Model Name
if(error_type == 0):
    #No Error
    model_name = "error_{0}".format(error_type)
elif(error_type == 1):
    model_name = "error_{0}_p_{1}".format(error_type,p)
elif(error_type == 2):
    #InnerDot error 
    model_name = "error_{0}_ul_{1}_uh_{2}_sigl_{3}_sigh_{4}".format(error_type,ul,uh,sigl,sigh)

if not(os.path.exists('models')):
    os.mkdir('models')
if not(os.path.exists('models/'+model_name)):
    os.mkdir('models/'+model_name)
    
    
# #gather all binary dense and binary convolution layers:
# binary_layers=[]
# for l in model.layers:
#     if isinstance(l,binary_dense) or isinstance(l,binary_conv):
#         binary_layers.append(l)

#Train
lr=0.01
opt = keras.optimizers.Adam(learning_rate=lr,decay=1e-6)#SGD(lr=lr,momentum=0.9,decay=1e-5)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])  


weights_path='models/'+model_name+'/'+model_name+'.h5'
cback=keras.callbacks.ModelCheckpoint(weights_path, monitor='val_accuracy', save_best_only=True,save_weights_only=True)


#Training
if keras.__version__[0]=='2':
    history=model.fit(X_train, y_train,batch_size=batch_size,validation_data=(X_val, y_val), verbose=2,epochs=epochs,callbacks=[cback])
if keras.__version__[0]=='1':
    history=model.fit(X_train, y_train,batch_size=batch_size,validation_data=(X_val, y_val), verbose=2,nb_epoch=epochs,callbacks=[cback])

#Save history    
dic={'hard':history.history}
foo=open('models/'+model_name+'.pkl','wb')
pickle.dump(dic,foo)
foo.close()

Epoch 1/100
1080/1080 - 13s - loss: 0.4406 - accuracy: 0.8786 - val_loss: 0.2317 - val_accuracy: 0.9335
Epoch 2/100
1080/1080 - 12s - loss: 0.2890 - accuracy: 0.9182 - val_loss: 0.2408 - val_accuracy: 0.9323
Epoch 3/100
1080/1080 - 11s - loss: 0.2406 - accuracy: 0.9315 - val_loss: 0.2098 - val_accuracy: 0.9400
Epoch 4/100
1080/1080 - 13s - loss: 0.2126 - accuracy: 0.9387 - val_loss: 0.1843 - val_accuracy: 0.9487
Epoch 5/100
1080/1080 - 11s - loss: 0.1911 - accuracy: 0.9452 - val_loss: 0.1704 - val_accuracy: 0.9517
Epoch 6/100
1080/1080 - 12s - loss: 0.1786 - accuracy: 0.9483 - val_loss: 0.1728 - val_accuracy: 0.9495
Epoch 7/100
1080/1080 - 12s - loss: 0.1662 - accuracy: 0.9513 - val_loss: 0.1718 - val_accuracy: 0.9525
Epoch 8/100
1080/1080 - 11s - loss: 0.1546 - accuracy: 0.9551 - val_loss: 0.1967 - val_accuracy: 0.9415
Epoch 9/100
1080/1080 - 11s - loss: 0.1502 - accuracy: 0.9564 - val_loss: 0.1943 - val_accuracy: 0.9458
Epoch 10/100
1080/1080 - 12s - loss: 0.1431 - accuracy: 0.9579 -

1080/1080 - 14s - loss: 0.0693 - accuracy: 0.9786 - val_loss: 0.1437 - val_accuracy: 0.9647
Epoch 80/100
1080/1080 - 13s - loss: 0.0750 - accuracy: 0.9767 - val_loss: 0.1441 - val_accuracy: 0.9625
Epoch 81/100
1080/1080 - 14s - loss: 0.0727 - accuracy: 0.9772 - val_loss: 0.1270 - val_accuracy: 0.9665
Epoch 82/100
1080/1080 - 14s - loss: 0.0708 - accuracy: 0.9785 - val_loss: 0.1255 - val_accuracy: 0.9662
Epoch 83/100
1080/1080 - 13s - loss: 0.0721 - accuracy: 0.9781 - val_loss: 0.1217 - val_accuracy: 0.9662
Epoch 84/100
1080/1080 - 15s - loss: 0.0711 - accuracy: 0.9778 - val_loss: 0.1384 - val_accuracy: 0.9660
Epoch 85/100
1080/1080 - 12s - loss: 0.0729 - accuracy: 0.9779 - val_loss: 0.1314 - val_accuracy: 0.9630
Epoch 86/100
1080/1080 - 12s - loss: 0.0713 - accuracy: 0.9776 - val_loss: 0.1423 - val_accuracy: 0.9623
Epoch 87/100
1080/1080 - 13s - loss: 0.0718 - accuracy: 0.9779 - val_loss: 0.1556 - val_accuracy: 0.9568
Epoch 88/100
1080/1080 - 11s - loss: 0.0712 - accuracy: 0.9777 - val

# Testing Script

In [51]:
#Test Based on Error Type
#Model Parameters
# 0 : No error, params ignored
# 1 : Flip error, only p used
# 2 : InnerProd error
train_error_type = 2 #can be 0 or 1 or 2
test_error_type = 2 # can be 0 1 or 2
train_flip_probability = 0.1

#Training InnerProd params
train_ul = 1
train_uh = 0.1
train_sigl = 0.01
train_sigh = 0.1


#Flip Probabiliy
p=0.1

# Test Parameters  
#InnerProd params
ul = 1
uh = 0.1
sigl = 0.01
sigh = 0.01

#Makes Model Name
if(train_error_type == 0):
    #No Error
    model_name = "error_{0}".format(train_error_type)
elif(train_error_type == 1):
    model_name = "error_{0}_p_{1}".format(train_error_type,train_flip_probability)
elif(train_error_type == 2):
    #InnerDot error 
    model_name = "error_{0}_ul_{1}_uh_{2}_sigl_{3}_sigh_{4}".format(train_error_type,train_ul,train_uh,train_sigl,train_sigh)    
    
weights_path = 'models/'+model_name+'/'+model_name+'.h5' #To Specify  

#weights_path = "" #To Specify

if(test_error_type == 0):
    #No Error
    model=make_MNIST_model(test_error_type, p = p, ul = ul, sigl = sigl, uh=uh, sigh = sigh )
    model.compile()

    #Load Weights
    model.load_weights(weights_path)
    
    score=model.evaluate(X_test,y_test,verbose=0)
    print("No Error, test loss was %0.4f, test accuracy was %0.4f"%(score[0],score[1]))

elif(test_error_type == 1):
    #Flip Error
    probs = np.logspace(-4, -1, 20)
    
    acc_1 = []
    for p in probs:
        
        model=make_MNIST_model(test_error_type, p = p, ul = ul, sigl = sigl, uh=uh, sigh = sigh )
        model.compile()
        
        #Load Weights
        model.load_weights(weights_path)
        
        
        
        score=model.evaluate(X_test,y_test,verbose=0)
        #print("With p= %0.4f, test loss was %0.4f, test accuracy was %0.4f" %(p,score[0],score[1]))
        print('%0.4f' %(score[1]))
        acc_1.append(score[1])
    
elif(test_error_type == 2):
    #InnerDot Error
    error_type = "InnerDot"
    
    ratio_l_h = np.linspace(1,80,20)
    acc_2 = []
    for ratio in ratio_l_h:
        print(ratio)
    for ratio in ratio_l_h:
        
        model = make_MNIST_model(test_error_type, p = p, ul = ul, sigl = sigl, uh=uh, sigh = ratio*sigl )
        
        #model.compile()
        lr=0.01
        opt = keras.optimizers.Adam(learning_rate=lr,decay=1e-6)#SGD(lr=lr,momentum=0.9,decay=1e-5)
        model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy']) 
        
        #Load Weights
        model.load_weights(weights_path)
        
        score=model.evaluate(X_test,y_test,verbose=0)
        #print("With ratio %0.4f, test loss was %0.4f, test accuracy was %0.4f"%(ratio,score[0],score[1]))
        print('%0.4f' %(score[1]))
        #print(score)
        acc_2.append(score[1])


1.0
5.157894736842105
9.31578947368421
13.473684210526315
17.63157894736842
21.789473684210527
25.94736842105263
30.105263157894736
34.26315789473684
38.421052631578945
42.578947368421055
46.73684210526316
50.89473684210526
55.05263157894737
59.21052631578947
63.36842105263158
67.52631578947368
71.6842105263158
75.84210526315789
80.0
0.9714
0.9699
0.9691
0.9685
0.9633
0.9575
0.9544
0.9463
0.9379
0.9218
0.9078
0.8877
0.8589
0.8193
0.7874
0.7466
0.7129
0.6569
0.6226
0.5587
